In [1]:
import pandas as pd
import geopandas as gpd
import geocoder
import nivapy
from shapely.geometry import Point
from shapely.ops import nearest_points

# Find nearest BGI

Quick Python example of identifying the nearest BGI point based on a user-supplied address.

## 1. Read locations for BGI

Line has a Google spreadsheet listing BGI projects in Oslo. This sheet includes lat/lon co-ordinates, but they are not consistently formatted. For the simple example here, I have tidied up the columns with spatial data and copied to Excel.

In [2]:
# Read data
bgi_xls = (r'C:\Data\James_Work\Staff\Line_B\iResponse\Vanniby_BGI-projects.xlsx')
bgi_df = pd.read_excel(bgi_xls, sheetname='Sheet1')
bgi_df

,Project name,lat,lon
0,"Årvolldammen, Hovinbekken og Glasbergbekken",59.947369,10.819674
1,Olaf Ryes plass,59.923463,10.758054
2,Bjølsen studentby,59.944849,10.760666
3,Hølaløkka ( må sjekkes mht til overvannsløsnin...,59.944849,10.760666
4,Nedre Foss park\n,59.922488,10.753258
5,OBOS Etterstadsletta ( mer privat?)\n,59.911184,10.799387
6,Lysaker - stasjonsaksens hovedtorg\n,59.915026,10.635812


In [3]:
# Convert to gdf
geometry = [Point(xy) for xy in zip(bgi_df.lon, bgi_df.lat)]
crs = {'init': 'epsg:4326'}
bgi_gdf = gpd.GeoDataFrame(bgi_df, crs=crs, geometry=geometry)
bgi_gdf

,Project name,lat,lon,geometry
0,"Årvolldammen, Hovinbekken og Glasbergbekken",59.947369,10.819674,POINT (10.819674 59.947369)
1,Olaf Ryes plass,59.923463,10.758054,POINT (10.7580535 59.9234632)
2,Bjølsen studentby,59.944849,10.760666,POINT (10.7606661 59.9448494)
3,Hølaløkka ( må sjekkes mht til overvannsløsnin...,59.944849,10.760666,POINT (10.7606661 59.9448494)
4,Nedre Foss park\n,59.922488,10.753258,POINT (10.753258 59.922488)
5,OBOS Etterstadsletta ( mer privat?)\n,59.911184,10.799387,POINT (10.799387 59.911184)
6,Lysaker - stasjonsaksens hovedtorg\n,59.915026,10.635812,POINT (10.635812 59.915026)


In [4]:
# Show BGI locs on map
map1 = nivapy.spatial.quickmap(bgi_df, 
                               lat_col='lat', 
                               lon_col='lon', 
                               popup='Project name',
                               tiles='openstreetmap')
map1

## 2. Get user input

In [5]:
# Address supplied by user
addr = 'NIVA, Gaustadalléen 21, 0349 Oslo'

# Geocode
g = geocoder.arcgis(addr)
lat, lon = g.latlng

print 'The co-ordinates of your address are:'
print (lon, lat)

The co-ordinates of your address are:
(10.71573419201851, 59.94240602885344)


## 3. Find nearest BGI

**Note:** The code below treats ellipsoidal (i.e. lat/lon) co-ordinates as Cartesian, which is not correct. However, given that all points are very close together, the errors involved should not affect determination of the nearest BGI.

In [6]:
# Build point from geocoded address
addr_loc = Point((lon, lat))

# Find nearest BGI
bgi_pts = bgi_gdf.unary_union
nearest = bgi_gdf.geometry == nearest_points(addr_loc, bgi_pts)[1]

print 'The nearest BGI to your address is:'
bgi_gdf[nearest]

The nearest BGI to your address is:


,Project name,lat,lon,geometry
4,Nedre Foss park\n,59.922488,10.753258,POINT (10.753258 59.922488)
